## Defining paths

In [1]:
dataset_root = '../../dataset/en-hi'
%env dataset_root = {dataset_root}
%env english_set_tokenized_train = {dataset_root}/final_train_norm_lf_1000_tk_moses.en
%env english_set_tokenized_val = {dataset_root}/final_val_norm_lf_1000_tk_moses.en
%env hindi_set_tokenized_train = {dataset_root}/final_train_norm_lf_1000_tk_indicnlp.hi
%env hindi_set_tokenized_val = {dataset_root}/final_val_norm_lf_1000_tk_indicnlp.hi
%env english_hindi_set_mixed = {dataset_root}/train.hi.en

env: dataset_root=../../dataset/en-hi
env: english_set_tokenized_train=../../dataset/en-hi/final_train_norm_lf_1000_tk_moses.en
env: english_set_tokenized_val=../../dataset/en-hi/final_val_norm_lf_1000_tk_moses.en
env: hindi_set_tokenized_train=../../dataset/en-hi/final_train_norm_lf_1000_tk_indicnlp.hi
env: hindi_set_tokenized_val=../../dataset/en-hi/final_val_norm_lf_1000_tk_indicnlp.hi
env: english_hindi_set_mixed=../../dataset/en-hi/train.hi.en


## Training shared BPE tokenizer

In [2]:
# concatenating both the datasets for training shared tokenizer
! cat $english_set_tokenized_train $hindi_set_tokenized_train > $english_hindi_set_mixed

In [3]:
# shuffling the dataset
!shuf $english_hindi_set_mixed -o $english_hindi_set_mixed.shuf
!rm $english_hindi_set_mixed

In [4]:
!yttm bpe --data $english_hindi_set_mixed.shuf --model preproc/tokenizer.BPE.32000.model --vocab_size 32000 --coverage 0.999

Training parameters
  input: ../../dataset/en-hi/train.hi.en.shuf
  model: preproc/tokenizer.BPE.32000.model
  vocab_size: 32000
  n_threads: 8
  character_coverage: 0.999
  pad: 0
  unk: 1
  bos: 2
  eos: 3

reading file...
learning bpe...
number of unique characters in the training data: 2874
number of deleted characters: 2764
number of unique characters left: 110
id: 1000=122+311              freq: 148556      subword: ▁these=▁the+se
id: 2000=580+184              freq: 60885       subword: ▁partic=▁part+ic
id: 3000=123+398              freq: 36705       subword: ▁मंद=▁म+ंद
id: 4000=144+234              freq: 25372       subword: ▁bad=▁b+ad
id: 5000=190+7                freq: 18524       subword: ilt=il+t
id: 6000=42+12                freq: 14388       subword: ws=w+s
id: 7000=2258+410             freq: 11478       subword: ▁लड़की=▁लड़+की
id: 8000=276+148              freq: 9472        subword: ▁जाया=▁जा+या
id: 9000=2128+354             freq: 7906        subword: ounter=oun+ter
id: 1

## Training NMT model

In [ ]:
!HYDRA_FULL_ERROR=1 python enc_dec_nmt.py \
  -cn aayn_base \
  model.preproc_out_dir=preproc \
  model.train_ds.use_tarred_dataset=true \
  model.train_ds.lines_per_dataset_fragment=1000000 \
  model.train_ds.num_batches_per_tarfile=200 \
  model.train_ds.use_tarred_dataset=true \
  model.train_ds.lines_per_dataset_fragment=1000000 \
  model.train_ds.num_batches_per_tarfile=200 \
  model.train_ds.src_file_name=$english_set_tokenized_train \
  model.train_ds.tgt_file_name=$hindi_set_tokenized_train \
  model.train_ds.tokens_in_batch=12500 \
  model.validation_ds.tokens_in_batch=8192 \
  model.validation_ds.src_file_name=$english_set_tokenized_val \
  model.validation_ds.tgt_file_name=$hindi_set_tokenized_val \
  model.encoder_tokenizer.vocab_size=32000 \
  model.decoder_tokenizer.vocab_size=32000 \
  ~model.test_ds \
  model.max_generation_delta=5 \
  model.shared_tokenizer=true \
  model.encoder_tokenizer.tokenizer_model=preproc/tokenizer.BPE.32000.model \
  model.decoder_tokenizer.tokenizer_model=preproc/tokenizer.BPE.32000.model \
  trainer.devices=[0,1,2,3] \
  ~trainer.max_epochs \
  +trainer.max_steps=150000 \
  +exp_manager.exp_dir=results \
  +exp_manager.create_checkpoint_callback=True \
  +exp_manager.checkpoint_callback_params.monitor=val_sacreBLEU \
  +exp_manager.checkpoint_callback_params.mode=max \
  +exp_manager.checkpoint_callback_params.save_top_k=5 \
  +trainer.val_check_interval=30000 

[NeMo W 2022-06-28 10:45:45 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-06-28 10:45:45 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo I 2022-06-28 10:45:45 enc_dec_nmt:108] 
    
    ************** Experiment configuration ***********
[NeMo I 2022-06-28 10:45:45 enc_dec_nmt:109] Config: name: AttentionIsAllYouNeed
    model:
      train_ds:
        src_file_name: ../../dataset/en-hi/final_train_norm_lf_1000_tk_moses.en
        tgt_file_name: ../../dataset/en-hi/final_train_norm_lf_1000_tk_indicnlp.hi
        use_tarred_dataset: true
        tar_files: null
        metadata_file: null
        lines_per_da